In [ ]:
import pandas
import numpy as np
import matplotlib.pyplot as plt
from google.colab import drive

In [ ]:
drive.mount('/content/gdrive')
csv_path = '/content/gdrive/My Drive/DL/YearPredictionMSD.txt.zip' # TODO - UPDATE ME WITH THE TRUE PATH!



Mounted at /content/gdrive
